<center><h2>MERCADOLIVRE DATA CHALLENGE 2019</h2><br></center>

<b>Autor:</b> Edwin Jahir Rueda Rojas<br>
<b>email:</b> ejrueda95g@gmail.com<br>

<b>Conjunto de treinamento:</b><br>
- Cada linha do conjunto de treinamento corresponde a um artigo no Mercado Livre. As colunas são as seguintes:
<ul>
    <li><b>Title</b><ul>
    <li>O título da publicação</li>
    </ul>
    <li><b>Language</b><ul>
    <li>O idioma do título. Pode ser espanhol ou português.</li>
    </ul>
    <li><b>Label quality</b><ul>
        <li>Pode ser confiável ou não confiável. Para os confiáveis, permanecemos tempo revisando essas etiquetas para minimizar a quantidade de erros (no entanto, eles também podem não ser 100% exatos). As linhas marcadas como não confiáveis não foram revisadas por nós, o que significa que a categoria foi escolhida pelo vendedor e, portanto, deve ser esperada uma taxa mais alta de erros de rotulagem.</li>
    </ul>
    <li><b>Category</b><ul>	
        <li>A categoria da publicação: variable target. O conjunto de categorias possíveis é o mesmo nos dois idiomas. Isso significa que o título de um celular será atribuído à categoria CELULARES, tanto em espanhol como em português.</li>
        </ul>        
</ul>
A distribuição de casos confiáveis no conjunto de treinamento não é uniforme. Ou seja, existem categorias que contêm muitos exemplos confiáveis, assim como outras que não contêm exemplos confiáveis.</p>

https://ml-challenge.mercadolibre.com/

<h3> Resultados </h3>
- A rede neuronal feita, tem um score de 91% nos dados de treinamento y un 72.20% nos dados de teste.
<br>
<h3> Conclusoes </h3>
- A rede poderia ter um melhor desempenho se a gente tivesse uma memoria RAM maior para poder ter um dicionário de palavras maior, a gente tinha 25gb de memoria RAM (google colab).

In [0]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time, sleep
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle, resample
from nltk.stem import SnowballStemmer
%matplotlib inline

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### A rede:
- Sao geradas duas redes, uma rede para o idioma espanhol e outra para o portugês.
- A arquitetura da rede e a elaboraçao do dataset é a mesma para os dois idiomas, por ende, neste notebook sao será vista uma metodología que a gente muda só ao mudar a variavél <b>idioma</b>.

In [0]:
idioma = "portuguese"

In [0]:
train = pd.read_csv("./drive/My Drive/Colab Notebooks/data/train.csv")
train_por = train.loc[train.language==idioma]
print(train_por.shape)
train_por.head()

(10000000, 4)


,title,label_quality,language,category
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
5,"4 Microaspersor Irrigação Ultra 7,20 Metros",unreliable,portuguese,IRRIGATION_SPRINKLERS
6,Raquete Clash 100 Tour - Nova,unreliable,portuguese,RACQUETS
7,"Kit Tripe Para Celular Ou Câmera Fotog 1,20m +...",unreliable,portuguese,CAMERA_TRIPODS


In [0]:
X_train = pd.DataFrame(columns=train_por.columns)
X_train.head()

,title,label_quality,language,category


In [0]:
len(pd.unique(train_por.category))

1576

In [0]:
train_por.shape

(10000000, 4)

In [0]:
targets = pd.unique(train_por.category.values)
len(targets)

1576

## Dataset
- O dataset contém 20.000.000 de dados, a mitade para português e a outra mitade para espahol.
- Como o dicionário de palavras é grande, a gente tem que reduzir a quantidade de mostras, por isso a gente só pega 200 mostras por categoria se nessa categoria os dados sao "reliable", se nao sao assim, a gente só pega 150 de forma aleatoria.
- O dicionário foi feito com as 5200 palavras mais repetidas.

In [37]:
t_i = time()
count = 0
for lbl in targets:
    aux = train_por.loc[train_por.label_quality=="reliable"]
    if aux.loc[aux.category==lbl].shape[0] != 0:
        X_train = X_train.append(aux.loc[aux.category==lbl].sample(200, replace=True)) #300
    else:
        aux2 = train_por.loc[train_por.label_quality=="unreliable"]
        X_train = X_train.append(aux2.loc[aux2.category==lbl].sample(150, replace=True)) #200
    count += 1 
    if count%100 == 0:
        t_f = time()
        print("count: ",count,"tiempo: ", np.round((t_f-t_i)/60, 5), "[min]")
        t_i = time()

count:  100 tiempo:  1.56836 [min]
count:  200 tiempo:  1.47169 [min]
count:  300 tiempo:  1.69253 [min]
count:  400 tiempo:  1.46935 [min]
count:  500 tiempo:  1.5956 [min]
count:  600 tiempo:  1.73726 [min]
count:  700 tiempo:  1.68368 [min]
count:  800 tiempo:  1.7674 [min]
count:  900 tiempo:  1.69649 [min]
count:  1000 tiempo:  1.87632 [min]
count:  1100 tiempo:  1.98181 [min]
count:  1200 tiempo:  1.90575 [min]
count:  1300 tiempo:  2.05629 [min]
count:  1400 tiempo:  2.14017 [min]
count:  1500 tiempo:  2.59912 [min]


In [38]:
print(X_train.shape)
X_train.to_csv("./drive/My Drive/Colab Notebooks/data/X_train_"+idioma+".csv")

(295400, 4)


In [0]:
X_train = pd.read_csv("./drive/My Drive/Colab Notebooks/data/X_train_"+idioma+".csv", index_col=0)

In [43]:
len(pd.unique(X_train.category))

1576

In [44]:
print(X_train.shape)
X_train.head()

(295400, 4)


,title,label_quality,language,category
18049546,Lava E Seca Midea Acqua Branco 8kg,reliable,portuguese,WASHING_MACHINES
9731921,Lavadora Roupas 12kg Suggar Lavamatic Semiaut....,reliable,portuguese,WASHING_MACHINES
11206431,Lavadora Electrolux De Roupas Automática 16kg ...,reliable,portuguese,WASHING_MACHINES
12707303,Lavadora Brastemp 11kg- Bwk11ab 220v,reliable,portuguese,WASHING_MACHINES
1599776,Lavadora De Roupas Latina La555 10kg - 110v,reliable,portuguese,WASHING_MACHINES


In [45]:
X_train = shuffle(X_train)
print(X_train.shape)
X_train.head()

(295400, 4)


,title,label_quality,language,category
15703885,"Encosto De Cabeça Com Tela E Dvd, Napoli Model...",reliable,portuguese,CAR_SCREENS
5679449,40%off Natura 2 Refis Chronos Antissinais 40g ...,reliable,portuguese,FACIAL_SKIN_CARE_PRODUCTS
11631957,Cabine Led/uv 48w Sun 8 Bivolt Unha Fibra Gel ...,reliable,portuguese,NAIL_DRYERS
7245629,Oculos Ogrife Solar Masculino Og 458-c Proteçã...,reliable,portuguese,SUNGLASSES
17926619,Envelope Vai-vem Plástico 26x35 772.0 Acrimet ...,reliable,portuguese,POLY_MAILERS


## Processamento dos dados
- No começo, todas as letras das palavras foram trocadas para minúsculas.<br>
- Depois, foram removidos todos os caracteres que nao sao ASCII e os números.<br>
- Só sao tomadas em conta as palavras que tem um comprimento maior a 3.<br>
- As palavras foram lematizadas e logo feito um <i>Stemmer</i> para assim pegar só as 5 primeiras letras da palavra.


In [0]:
def processing(doc):
    return doc.lower()

In [0]:
import re, unicodedata, inflect

def token(doc):
    words = nltk.word_tokenize(doc, language=idioma)
    lemmatizer = nltk.WordNetLemmatizer()
    r = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_word = re.sub(r'[^\w\s]', '', new_word)
        new_word = re.sub(r'\d','',new_word)
        if len(new_word.replace(" ","")) > 3:
            new_word = lemmatizer.lemmatize(new_word, pos='v')
            new_word = lemmatizer.lemmatize(new_word, pos='n')
            new_word = SnowballStemmer(idioma).stem(new_word)
            r.append(new_word[:5])
    return r

In [70]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
token("bom, dias caras vendemos, vendese, vender")

['dia', 'car', 'vend', 'vende', 'vend']

## Criaçao do DataSet
- Para a criaçao do dataset, foi usada a funçao <i>CountVectorizer</i> de sklearn, ela gera um DataSet a partir do contagem de palavras, a gente pegou as 5200 palavras mais repetidas.

In [0]:
text = X_train.title.values

vectorizer = CountVectorizer(stop_words=[sw[:4] for sw in nltk.corpus.stopwords.words(idioma)], strip_accents="unicode",
                             preprocessor=processing, decode_error="ignore", analyzer="word", max_features=5200, tokenizer=token) #3500

X_train_por = vectorizer.fit_transform(text)
print("número de features: ", len(vectorizer.get_feature_names()))
encoder = OrdinalEncoder()
y_train_por = X_train.category.values.reshape(-1, 1)
encoder.fit(y_train_por)
y_por_enc = encoder.transform(y_train_por)
print(X_train_por.shape, y_por_enc.shape)

In [ ]:
vectorizer.get_feature_names(), len(vectorizer.get_feature_names())

In [0]:
X_train_por = X_train_por.toarray()
print(X_train_por.shape)

(421000, 5200)


In [0]:
X_train_por.shape

## A rede
- Foi feita uma rede neuronal com 4 camadas de densidade e 2 camadas de eliminaçao com um modelo sequencial feito com tensorflow.keras.

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop

In [0]:
model = Sequential()
model.add(Dense(600, input_shape=(X_train_por.shape[1],), name="dense1", activation="sigmoid"))
model.add(Dropout(.1))
model.add(Dense(600,  name="dense2", activation="tanh"))
model.add(Dropout(.1))
model.add(Dense(800,  name="dense3", activation="tanh"))
model.add(Dense(len(np.unique(y_por_enc)), activation="softmax"))

In [0]:
model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
X_train_por.shape

(295400, 5200)

In [54]:
y_binary = to_categorical(y_por_enc)
print(X_train_por.shape, y_binary.shape)
model.fit(X_train_por, y_binary, batch_size=64, epochs=30)

(295400, 5200) (295400, 1576)
Epoch 1/30
295400/295400 [==============================] - 216s 730us/sample - loss: 4.3745 - acc: 0.3404
Epoch 2/30
295400/295400 [==============================] - 215s 728us/sample - loss: 1.8644 - acc: 0.6993
Epoch 3/30
295400/295400 [==============================] - 215s 728us/sample - loss: 1.3850 - acc: 0.7744
Epoch 4/30
295400/295400 [==============================] - 215s 728us/sample - loss: 1.1893 - acc: 0.8048
Epoch 5/30
295400/295400 [==============================] - 213s 721us/sample - loss: 1.0807 - acc: 0.8222
Epoch 6/30
295400/295400 [==============================] - 213s 720us/sample - loss: 1.0042 - acc: 0.8337
Epoch 7/30
295400/295400 [==============================] - 214s 723us/sample - loss: 0.9532 - acc: 0.8424
Epoch 8/30
295400/295400 [==============================] - 214s 725us/sample - loss: 0.9129 - acc: 0.8489
Epoch 9/30
295400/295400 [==============================] - 213s 722us/sample - loss: 0.8808 - acc: 0.8538
Epoch 1

In [0]:
model.save('./drive/My Drive/Colab Notebooks/data/my_model' + idioma + '.h5')

In [0]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('./drive/My Drive/Colab Notebooks/data/my_model' + idioma + '.h5')
# summarize model.
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 600)               3120600   
_________________________________________________________________
dropout (Dropout)            (None, 600)               0         
_________________________________________________________________
dense2 (Dense)               (None, 600)               360600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 600)               0         
________________________________

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 600)               3120600   
_________________________________________________________________
dropout (Dropout)            (None, 600)               0         
_________________________________________________________________
dense2 (Dense)               (None, 600)               360600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense3 (Dense)               (None, 800)               480800    
_________________________________________________________________
dense (Dense)                (None, 1574)              1260774   
Total params: 5,222,774
Trainable params: 5,222,774
Non-trainable params: 0
______________________________________________

In [56]:
y_binary = to_categorical(y_por_enc)
model.evaluate(X_train_por.toarray(), y_binary)

295400/295400 [==============================] - 67s 227us/sample - loss: 0.5041 - acc: 0.9117


[0.5041195535061019, 0.9117231]

In [57]:
data_test = pd.read_csv("./drive/My Drive/Colab Notebooks/data/test.csv", index_col=0)
data_test.head()

,title,language
id,,
0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese
1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese
2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese
3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese
4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese


In [58]:
new_dtest = data_test.loc[data_test.language==idioma]
new_dtest.head()

,title,language
id,,
0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...,portuguese
1,Trocador De Fraldas Fisher Price Feminino Rosa...,portuguese
2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...,portuguese
3,Amortecedor Mola Batente D Dir New Civic 14 - ...,portuguese
4,Cadeirinha De Carro Bebê Princesa Princess 9 A...,portuguese


In [0]:
text_test = new_dtest.title.values

X_test_por = vectorizer.transform(text_test)

In [0]:
y_test_por = model.predict_classes(X_test_por.toarray())

In [61]:
y_test_por

array([ 449,  120,   63, ...,   82, 1521,  458])

In [62]:
y_test_des = encoder.inverse_transform(y_test_por.reshape(-1, 1))
y_test_des

array([['DIAPER_BAGS'],
       ['BABY_CHANGING_PADS'],
       ['AUTOMOTIVE_BLOWER_HEATER_MOTORS'],
       ...,
       ['AUTOMOTIVE_HEADLIGHTS'],
       ['WALKIE_TALKIES'],
       ['DINING_SETS']], dtype=object)

In [63]:
submission = pd.read_csv("./drive/My Drive/Colab Notebooks/data/sample_submission16.csv",index_col=0)
submission.iloc[:25]

,category
id,
0,DIAPER_BAGS
1,BABY_DIAPERS
2,ENGINE_COOLING_FAN_MOTORS
3,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,BABY_CAR_SEATS
5,HAND_BRAKE_CABLES
6,NETBOOKS
7,VEHICLE_LED_BULBS
8,CD_AND_DVD_PLAYERS


In [0]:
submission.loc[new_dtest.index] = y_test_des

In [65]:
submission.loc[new_dtest.index]

,category
id,
0,DIAPER_BAGS
1,BABY_CHANGING_PADS
2,AUTOMOTIVE_BLOWER_HEATER_MOTORS
3,AUTOMOTIVE_SHOCK_ABSORBER_BUMP_STOPS
4,BABY_CAR_SEATS
5,HAND_BRAKE_CABLES
6,DESKTOP_COMPUTERS
7,XENON_KITS
8,MARTIAL_ARTS_FOOT_GUARDS


In [0]:
submission.to_csv("./drive/My Drive/Colab Notebooks/data/sample_submission17.csv")